#### This Code is the base of the current running Cate ML Model

In [1]:
import pandas as pd
pd.options.plotting.backend = "plotly"

import plotly.express as px
import plotly.graph_objects as go

import numpy as np

from make_new_features import get_df_with_features
from get_krx_value import get_krx_mean

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

from xgboost import plot_importance
import matplotlib.pyplot as plt

C:\Users\stop2\AppData\Local\Temp\ipykernel_23496\236577770.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


#### Preparing Price Dataset

In [2]:
SHORT_PERIOD=5
MID_PERIOD=20
LONG_PERIOD=60

try :
    df_with_feats = pd.read_pickle(f"./df_with_feats_f_20130101_t_20230919_{SHORT_PERIOD}_{MID_PERIOD}_{LONG_PERIOD}_with_min_max_ratio.pkl")
    # df_with_feats['date'] = df_with_feats['date'].dt.strftime("%Y-%m-%d")
except :
    # filtering : Normal Stocks, Not SPAC
    df_price = (
        pd.read_pickle("./df_price_price_only_f_20130101_t_20230919.pkl")
        .loc[lambda df : df["code"].str[5]== "0"]
        # .loc[lambda df : ~df["name"].str.contains("스펙")]
        # .loc[lambda df : ~df["name"].str.contains("스팩")]    
    )

    df_price['date'] = df_price['date'].dt.strftime("%Y-%m-%d")

    try:
        df_krx = pd.read_pickle("./df_krx.pkl")
    except :
        df_krx = get_krx_mean()
        df_krx.to_pickle("./df_krx.pkl")

    df_price = df_price.merge(
        df_krx,
        on='date'
    )
    
    df_with_feats = get_df_with_features(
        df_price, SHORT_PERIOD=SHORT_PERIOD, MID_PERIOD=MID_PERIOD, LONG_PERIOD=LONG_PERIOD)
    df_with_feats.to_pickle(f"./df_with_feats_f_20130101_t_20230919_{SHORT_PERIOD}_{MID_PERIOD}_{LONG_PERIOD}_with_min_max_ratio.pkl")

In [3]:
df_with_feats.columns

Index(['date', 'open', 'high', 'low', 'close', 'volume', 'change', 'code',
       'name', 'marcap', 'kospi', 'kosdaq', 'weighted_price', 'vol_x_price',
       'vol_x_price_sma_long', 'vol_x_price_sma_mid', 'vol_x_price_sma_short',
       'change_high', 'change_low', 'change_open', 'close_std_short',
       'close_std_mid', 'close_std_long', 'close_mean_short', 'close_mean_mid',
       'close_mean_long', 'close_min_ratio_short', 'close_min_ratio_mid',
       'close_min_ratio_long', 'close_max_ratio_short', 'close_max_ratio_mid',
       'close_max_ratio_long', 'open_std_short', 'open_std_mid',
       'open_std_long', 'high_std_short', 'high_std_mid', 'high_std_long',
       'low_std_short', 'low_std_mid', 'low_std_long', 'vol_zero_count_short',
       'vol_zero_count_mid', 'vol_zero_count_long', 'close_change_p_short',
       'close_change_p_mid', 'close_change_p_long', 'w_price_vol_corr_long',
       'w_price_vol_corr_mid', 'w_price_vol_corr_short', 'krx_corr_short',
       'krx_corr_mi

In [4]:
df_with_feats.shape

(5129630, 63)

## ML

In [5]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [6]:
df_with_feats_ml = (
    df_with_feats
    .assign(
        vol_x_price_sma_long_to_mid = lambda df : df.vol_x_price_sma_long / df.vol_x_price_sma_mid,
        vol_x_price_sma_mid_to_short = lambda df : df.vol_x_price_sma_mid / df.vol_x_price_sma_short,
    )
    .loc[lambda df : df.code.str[5] == '0']
    .loc[lambda df : ~df.name.str.contains("스펙")]
    .loc[lambda df : ~df.name.str.contains("스팩")]

    .loc[lambda df : df.marcap > 500_00_000_000]
    
    .loc[lambda df : df["vol_x_price_sma_mid"] >  0.75e+08]  # 0.75e+08
    .loc[lambda df : df["vol_x_price_sma_mid"] <  3.5e+08]  # 3.5e+08
    .loc[lambda df : df["vol_zero_count_mid"] == 0]
    .loc[lambda df : df["change"] < 0.29]
    .loc[lambda df : df["volume"] > 0]

    # .loc[lambda df : df.close > 1000]
)

In [2]:
# For fixed length train dataset

import exchange_calendars as xcals
krx_cal = xcals.get_calendar("XKRX")

max_date = '2023-09-19'
# max_date = '2024-01-19'
start_date = '2015-01-02'

finish = False
dates = []

len_of_train = 200
gap_from_last_train_date = 7 # This number depends on the target ( rtn_5 -> 7, rnt_20 -> 22)
len_of_pred = 20 # The length of pred for 1 model update

while not finish :
    train_end = krx_cal.sessions_window(start_date, len_of_train)[-1].strftime("%Y-%m-%d")
    oos_start = krx_cal.sessions_window(train_end, gap_from_last_train_date)[-1].strftime("%Y-%m-%d")
    oos_end = krx_cal.sessions_window(oos_start, len_of_pred)[-1].strftime("%Y-%m-%d")

    if oos_end > max_date :
        dates.append(
            (start_date, train_end, oos_start, max_date)
        )
        finish = True

    else :
        dates.append(
            (start_date, train_end, oos_start, oos_end)
        )

        start_date = krx_cal.sessions_window(start_date, len_of_pred)[-1].strftime("%Y-%m-%d")

### Feats

In [9]:
feats_rtn_5 = [
    'vol_x_price_sma_short',
    'close_std_mid',
    'close_std_long',
    'open_std_short',
    'open_std_mid',
    'open_std_long',
    'high_std_short',
    'high_std_mid',
    'low_std_short',
    'close_change_p_long',
    'krx_corr_mid',
    'krx_corr_long',
    'krx_change_std_short',
    'krx_change_std_mid',
    'krx_change_std_long',
    'w_price_vol_corr_long',
    'w_price_vol_corr_mid',

    'close_mean_short',
    'close_mean_mid',
    'close_mean_long',
    'close_min_ratio_short',
    'close_min_ratio_mid',
    'close_min_ratio_long',
    'close_max_ratio_short',
    'close_max_ratio_mid',
    'close_max_ratio_long',       
]

target_rtn_5 = 'rtn_5'

In [10]:
feats_rtn_4 = [
    'vol_x_price_sma_short',
    'vol_x_price_sma_mid_to_short',
    'change_high',
    'change_low',
    'change_open',
    'close_std_short',
    'close_std_mid',
    'close_std_long',
    'open_std_short',
    'open_std_mid',
    'high_std_short',
    'high_std_long',
    'close_change_p_long',
    'krx_corr_mid',
    'krx_corr_long',
    'krx_change_std_short',
    'krx_change_std_mid',
    'w_price_vol_corr_long',
    'w_price_vol_corr_mid',

    'close_mean_short',
    'close_mean_mid',
    'close_mean_long',
    'close_min_ratio_short',
    'close_min_ratio_mid',
    'close_min_ratio_long',
    'close_max_ratio_short',
    'close_max_ratio_mid',
    'close_max_ratio_long',
]

target_rtn_4 = 'rtn_4'

In [11]:
feats_rtn_3 = [
    'w_price_vol_corr_long',
    'vol_x_price_sma_mid_to_short',
    'w_price_vol_corr_short',
    'vol_x_price_sma_short',
    'w_price_vol_corr_mid',
    'close_change_p_long',
    'high_std_long',
    'krx_corr_mid',
    'krx_corr_long',
    'change_low',
    'open_std_short',
    'close_std_long',
    'high_std_short',
    'open_std_mid',
    'close_std_short',
    'change_high',
    'krx_change_std_short',
    'krx_change_std_mid',
    'close_std_mid',

    'close_mean_short',
    'close_mean_mid',
    'close_mean_long',
    'close_min_ratio_short',
    'close_min_ratio_mid',
    'close_min_ratio_long',
    'close_max_ratio_short',
    'close_max_ratio_mid',
    'close_max_ratio_long',
]

target_rtn_3 = 'rtn_3'

#### Create df_oos_with_proba

In [13]:
def run_ml_multi_seed(features, idx0, idx1, fixed, target):

    feats = features
    
    THRES_OF_PROBA = 0.5

    rtn_in_period = []
    l_df = []

    if fixed:
        print("fixed")
    else :
        print("float")

    for train_start, train_end, oos_start, oos_end in dates[idx0 : idx1]:

        df_train_set = df_with_feats_ml.loc[lambda df : df.date >= train_start].loc[lambda df : df.date < train_end]
        df_oos_set = df_with_feats_ml.loc[lambda df : df.date >= oos_start].loc[lambda df : df.date < oos_end]

        df_train_set_ = (
            df_train_set
            .replace([-np.inf, np.inf], np.nan)
            .dropna(subset=feats)
            .reset_index(drop=True)
        )

        if fixed:
            THRES_OF_TRUE = 0.03
        else :
            THRES_OF_TRUE = df_train_set_[target].quantile(0.75)
        
        # print(THRES_OF_TRUE)

        df_oos_set_ = (
            df_oos_set      
            .replace([-np.inf, np.inf], np.nan)
            .dropna(subset=feats)
            .reset_index(drop=True)
        )

        df_train_set_["target"] = (
                df_train_set_[target].apply(lambda x : 1 if x > THRES_OF_TRUE else 0)
            )

        feature_df = df_train_set_[feats]
        target_df = df_train_set_["target"]

        feature_df_oos = df_oos_set_[feats]

        doos = xgb.DMatrix(feature_df_oos)        

        if df_oos_set_.shape[0] > 0:
            
            col_list = []
            for id, seed in enumerate([6, 13, 5, 2]):

                X_train, X_valid, y_train, y_valid = train_test_split(
                    feature_df, target_df, test_size=0.25, random_state=seed
                )

                # Create DMatrix for training and validation data
                dtrain = xgb.DMatrix(X_train, label=y_train)
                dvalid = xgb.DMatrix(X_valid, label=y_valid)

                evals = [(dtrain, 'train'), (dvalid, 'eval')]

                params = {
                    'max_depth': 5, # Adjust based on your dataset : originally 5 --> try 8
                    'eta': 0.05,     # Learning rate : originally 0.05
                    'objective': 'binary:logistic',
                    'eval_metric': 'logloss',  # Or use 'auc', 'error', etc. based on your problem
                    'random_state': 42,
                }

                model = xgb.train(
                    params, dtrain,
                    1000,
                    evals=evals,
                    early_stopping_rounds=50,
                    verbose_eval=False
                )

                df_oos_set_[f"pred_proba_{id}"] = model.predict(doos)
                col_list.append(f"pred_proba_{id}")
            
            df_oos_set_["proba_mean"] = df_oos_set_[col_list].mean(axis=1)
            df_oos_set_["proba_max"] = df_oos_set_[col_list].max(axis=1)

            l_df.append(df_oos_set_)

            sr_selected_mean_mean = (
                df_oos_set_
                .loc[lambda df : df['proba_mean'] > THRES_OF_PROBA]
                .sort_values('proba_mean', ascending=False)
                .groupby('date')
                .head(5)
                .reset_index()
                .groupby('date')[target]
                .mean()
                -0.0023
            )       

            sr_selected_mean_max = (
                df_oos_set_
                .loc[lambda df : df['proba_mean'] > THRES_OF_PROBA]
                .sort_values('proba_max', ascending=False)
                .groupby('date')
                .head(5)
                .reset_index()
                .groupby('date')[target]
                .mean()
                -0.0023
            )      

            sr_selected_max_mean = (
                df_oos_set_
                .loc[lambda df : df['proba_max'] > THRES_OF_PROBA]
                .sort_values('proba_mean', ascending=False)
                .groupby('date')
                .head(5)
                .reset_index()
                .groupby('date')[target]
                .mean()
                -0.0023
            )    

            sr_selected_max_max = (
                df_oos_set_
                .loc[lambda df : df['proba_max'] > THRES_OF_PROBA]
                .sort_values('proba_max', ascending=False)
                .groupby('date')
                .head(5)
                .reset_index()
                .groupby('date')[target]
                .mean()
                -0.0023
            )     

            rtn_in_period.append(
                {
                    'date' : oos_end,
                    "return_mean_mean" : sr_selected_mean_mean.sum()/(int(target.split('_')[-1]) + 1),
                    'return_mean_max' : sr_selected_mean_max.sum()/(int(target.split('_')[-1]) + 1),
                    'return_max_mean' : sr_selected_max_mean.sum()/(int(target.split('_')[-1]) + 1),
                    'return_max_max' : sr_selected_max_max.sum()/(int(target.split('_')[-1]) + 1),
                    'date_n' : sr_selected_mean_max.shape[0]
                }
            )

        else :

        
            # cumsum += 0
            rtn_in_period.append(
                {
                    'date' : oos_end,
                    "return_mean_mean" : 0,
                    'return_mean_max' : 0,
                    'return_max_mean' : 0,
                    'return_max_max' : 0,
                    'date_n' : 0
                }
            )
        
        
    return rtn_in_period, l_df

In [ ]:
for feat_var, target_var in [(feats_rtn_3, target_rtn_3), (feats_rtn_4, target_rtn_4), (feats_rtn_5, target_rtn_5)]: #

    feats = feat_var
    target = target_var

    for tf in [True, False]:

        print(target)

        oos_test_result, dfs = run_ml_multi_seed(feats, 0, -1, tf, target)
        df_oos_result = pd.DataFrame(oos_test_result)
        df_oos_result = df_oos_result.set_index('date')

        if tf:
            df_oos_with_proba_fixed = pd.concat(dfs)
            df_oos_with_proba_fixed.to_pickle(f"df_oos_with_proba_fixed_{target}.pkl")
        else:
            df_oos_with_proba_float = pd.concat(dfs)
            df_oos_with_proba_float.to_pickle(f"df_oos_with_proba_float_{target}.pkl")

        for col in df_oos_result.columns:
            sr = (df_oos_result[col]+1)
            sr_org = df_oos_result[col]

            print(col)
            print(f"Prod : {round(sr.cumprod()[-1],2)}")
            print(f"Prod Max : {round(sr.cumprod().max(),2)}")
            print(f"Prod Min : {round(sr.cumprod().min(),2)}")
            print(f"min : {round(sr_org.min(),3)}")
            print(f"std : {round(sr_org.std(),3)}")
            print(f"mean : {round(sr_org.mean(),3)}")
            print('\n')


#### Get Best Thresholds for Proba

In [4]:
def objective(trial, df_proba, rtn_n):

    thres01 = trial.suggest_float("thres_for_max", 0.3, 0.9, step=0.01)
    thres02 = trial.suggest_float("thres_for_mean", 0.3, 0.9, step=0.01)
    # thres03 = trial.suggest_float("thres_mean_max", 0.3, 0.9, step=0.01)
    # thres04 = trial.suggest_float("thres_mean_mean", 0.3, 0.9, step=0.01)

    l_rtn_p = []

    for date in dates:

        df_ = df_proba.loc[lambda df : df.date >= date[2]].loc[lambda df : df.date < date[3]]


        df1 = (
            df_
            .loc[lambda df : df['proba_max'] > thres01]
            .sort_values('proba_max', ascending=False)
            .groupby('date')
            .head(3)
            .reset_index()
        )

        df2 = (
            df_
            .loc[lambda df : df['proba_max'] > thres01]
            .sort_values('proba_mean', ascending=False)
            .groupby('date')
            .head(3)
            .reset_index()
        )

        df3 = (
            df_
            .loc[lambda df : df['proba_mean'] > thres02]
            .sort_values('proba_max', ascending=False)
            .groupby('date')
            .head(3)
            .reset_index()
        )

        df4 = (
            df_
            .loc[lambda df : df['proba_mean'] > thres02]
            .sort_values('proba_mean', ascending=False)
            .groupby('date')
            .head(3)
            .reset_index()
        )

        sr = (
            pd.concat([df1, df2, df3, df4]).drop_duplicates(subset=['date','code']) #df1, df2, 
            .groupby('date')[rtn_n]
            .mean()
            -0.0023
        )

        n = int(rtn_n.split('_')[1]) + 1

        l_rtn_p.append(
            {
                'date' : date[3],
                'rtn_p' : (sr/n).sum(),
                'days' : pd.concat([df1, df2, df3, df4]).drop_duplicates(subset=['date','code'])['date'].unique().__len__()
            }        
        )

    sr_final = (
        pd.DataFrame(l_rtn_p)
        .set_index('date')
        ['rtn_p']
        # +1
    )

    print(
        pd.DataFrame(l_rtn_p)['days'].sum()
    )

    return (sr_final+1).cumprod()[-1]


In [6]:
import optuna

In [7]:
# Optuna Strudy Run
def run_optuna(name, df_, rtn_n):

    study = optuna.create_study(
            direction="maximize",
            # direction="minimize",
            study_name=name,
        )

    study.optimize(
        lambda trial : objective(trial, df_, rtn_n),
        n_trials=250,
        show_progress_bar=True,
    )

    # Create study result df & Save to file
    df_study_result = (
        study.trials_dataframe()
        .rename(columns = lambda x : x.replace("params_", ""))
    )    
    df_study_result.to_pickle(f"./study_result_20240214_{name}.pkl")

In [8]:
for rtn_n in ['rtn_3', 'rtn_4', 'rtn_5']:
    for tf in [True, False]:

        type_str = 'fixed' if tf else 'float'

        df_with_proba = pd.read_pickle(f"df_oos_with_proba_{type_str}_{rtn_n}.pkl")
        run_optuna(f"best_combi_of_thres_{type_str}_{rtn_n}", df_with_proba, rtn_n)

[I 2024-02-15 04:50:54,995] A new study created in memory with name: best_combi_of_thres_fixed_rtn_3
Best trial: 0. Best value: 6.10991:   0%|          | 1/250 [00:07<31:39,  7.63s/it]

686
[I 2024-02-15 04:51:02,624] Trial 0 finished with value: 6.109908615495038 and parameters: {'thres_for_max': 0.75, 'thres_for_mean': 0.77}. Best is trial 0 with value: 6.109908615495038.


Best trial: 1. Best value: 11.6091:   1%|          | 2/250 [00:15<30:57,  7.49s/it]

902
[I 2024-02-15 04:51:10,016] Trial 1 finished with value: 11.609112763434212 and parameters: {'thres_for_max': 0.69, 'thres_for_mean': 0.76}. Best is trial 1 with value: 11.609112763434212.


Best trial: 1. Best value: 11.6091:   1%|          | 3/250 [00:22<30:44,  7.47s/it]

...

1710
[I 2024-02-15 08:06:54,608] Trial 247 finished with value: 18.754856805604202 and parameters: {'thres_for_max': 0.53, 'thres_for_mean': 0.8899999999999999}. Best is trial 17 with value: 20.345378790908132.


Best trial: 17. Best value: 20.3454: 100%|█████████▉| 249/250 [32:12<00:07,  7.74s/it]

1739
[I 2024-02-15 08:07:02,378] Trial 248 finished with value: 19.97058381364566 and parameters: {'thres_for_max': 0.51, 'thres_for_mean': 0.8999999999999999}. Best is trial 17 with value: 20.345378790908132.


Best trial: 17. Best value: 20.3454: 100%|██████████| 250/250 [32:20<00:00,  7.76s/it]

1698
[I 2024-02-15 08:07:10,196] Trial 249 finished with value: 18.515870344366252 and parameters: {'thres_for_max': 0.54, 'thres_for_mean': 0.8600000000000001}. Best is trial 17 with value: 20.345378790908132.


In [9]:
def get_df_rtn_result(df_with_proba, thres_for_max, thres_for_mean, rtn):
    l_rtn_p = []

    n = int(rtn.split('_')[1])+1

    for date in dates:

        df_ = df_with_proba.loc[lambda df : df.date >= date[2]].loc[lambda df : df.date < date[3]]


        df1 = (
            df_
            .loc[lambda df : df['proba_max'] > thres_for_max]
            .sort_values('proba_max', ascending=False)
            .groupby('date')
            .head(3)
            .reset_index()
        )

        df2 = (
            df_
            .loc[lambda df : df['proba_max'] > thres_for_max]
            .sort_values('proba_mean', ascending=False)
            .groupby('date')
            .head(3)
            .reset_index()
        )

        df3 = (
            df_
            .loc[lambda df : df['proba_mean'] > thres_for_mean]
            .sort_values('proba_max', ascending=False)
            .groupby('date')
            .head(3)
            .reset_index()
        )

        df4 = (
            df_
            .loc[lambda df : df['proba_mean'] > thres_for_mean]
            .sort_values('proba_mean', ascending=False)
            .groupby('date')
            .head(3)
            .reset_index()
        )

        sr = (
            pd.concat([df1, df2, df3, df4]).drop_duplicates(subset=['date','code']) #df1, df2, 
            .groupby('date')[rtn]
            .mean()
            -0.0023
        )

        l_rtn_p.append(
            {
                'date' : date[3],
                'rtn_p' : (sr/n).sum(),
                'days' : pd.concat([df1, df2, df3, df4]).drop_duplicates(subset=['date','code'])['date'].unique().__len__()
            }        
        )

    return (
        pd.DataFrame(l_rtn_p)
        .set_index('date')
        ['rtn_p']
    )

In [10]:
from scipy import stats

In [11]:
def get_rsquare(df_with_proba, rtn, row):
    
    sr_final = get_df_rtn_result(df_with_proba, row.thres_for_max, row.thres_for_mean, rtn)

    x = sr_final.cumsum()
    y = range(0, len(x))

    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

    print(f"R-squared: {r_value**2}")
    return r_value**2

#### Add R-Squared

In [14]:
typeStr = None

for useFixed in [False]: #, False
    if useFixed:
        typeStr = 'fixed'
    else :
        typeStr = 'float'

    for rtn in ['rtn_4','rtn_5']:
        df_with_proba = pd.read_pickle(f"df_oos_with_proba_{typeStr}_{rtn}.pkl")
        df_study_result = pd.read_pickle(f"study_result_20240214_best_combi_of_thres_{typeStr}_{rtn}.pkl")

        df_study_result = df_study_result.drop_duplicates(subset=['thres_for_max','thres_for_mean']).sort_values("value", ascending=False).head(100)

        df_study_result['r_square'] = df_study_result.apply(lambda row : get_rsquare(df_with_proba, rtn, row), axis=1 )

        
        df_study_result.to_pickle(f"study_result_add_r2_{typeStr}_{rtn}.pkl")
        

R-squared: 0.985067427504975
R-squared: 0.984798108413385
R-squared: 0.984981256311024
R-squared: 0.9847080912539343
R-squared: 0.9845946094907247
R-squared: 0.9848633217506
R-squared: 0.984588497016251
R-squared: 0.9843805015635497
R-squared: 0.9844302897037166
R-squared: 0.9840598945765093
R-squared: 0.9844495445629887
R-squared: 0.9844650896764864
R-squared: 0.9841427640142376
R-squared: 0.9857310239214223
R-squared: 0.9844345291740302
R-squared: 0.9842557188027857
R-squared: 0.9858814133649335
R-squared: 0.9841305280902346
R-squared: 0.9839403194542802
R-squared: 0.9856464709550237
R-squared: 0.9840914407504221
R-squared: 0.9844106371429298
R-squared: 0.9842697556161553
R-squared: 0.9842807826288955
R-squared: 0.9855298451287708
R-squared: 0.9839615697341503
R-squared: 0.9843042020905451
R-squared: 0.9839738880292385
R-squared: 0.9850518260299128
R-squared: 0.984354653564354
R-squared: 0.9854024132344584
R-squared: 0.9840566608957383
R-squared: 0.9839467443899118
R-squared: 0.98496

#### Get Bset R2 Results

In [25]:
l_result = []

for typeStr in ['fixed', 'float']:

    for rtn in ['rtn_3', 'rtn_4', 'rtn_5']:

        print(f"{rtn}_{typeStr}")
        df_with_proba = pd.read_pickle(f"df_oos_with_proba_{typeStr}_{rtn}.pkl")
        df_study_result_with_rs = pd.read_pickle(f"study_result_add_r2_{typeStr}_{rtn}.pkl")

        max_row = df_study_result_with_rs.sort_values('r_square', ascending=False).head(1)
        print(max_row)
        thres_for_max = max_row['thres_for_max'].iloc[0]
        thres_for_mean = max_row['thres_for_mean'].iloc[0]

        sr_final = get_df_rtn_result(df_with_proba, thres_for_max, thres_for_mean, rtn)
        sr_final.name = f"{rtn}_{typeStr}"
        
        l_result.append(
            sr_final.rename({'rtn_p': f"{rtn}_{typeStr}"})
        )
        print('\n')

rtn_3_fixed
     number      value             datetime_start          datetime_complete  \
230     230  14.149911 2024-02-15 05:20:33.137309 2024-02-15 05:20:40.877201   

                  duration  thres_for_max  thres_for_mean     state  r_square  
230 0 days 00:00:07.739892           0.53             0.4  COMPLETE  0.986495  


rtn_4_fixed
    number      value             datetime_start          datetime_complete  \
30      30  22.543772 2024-02-15 06:00:41.326107 2024-02-15 06:00:49.022705   

                 duration  thres_for_max  thres_for_mean     state  r_square  
30 0 days 00:00:07.696598           0.53            0.35  COMPLETE  0.987656  


rtn_5_fixed
     number      value             datetime_start          datetime_complete  \
106     106  22.785744 2024-02-15 07:15:44.283836 2024-02-15 07:15:52.124981   

                  duration  thres_for_max  thres_for_mean     state  r_square  
106 0 days 00:00:07.841145           0.35            0.48  COMPLETE  0.990058  



In [16]:
df_result_opt = pd.concat(l_result, axis=1)

#### Final Result Summary
- Set All to 'fixed'
- rtn_3 : thres_for_max 0.53 / thres_for_mean 0.4 / value 14.149
- rtn_4 : thres_for_max 0.53 / thres_for_mean 0.35 / value 22.543
- rtn_5 : thres_for_max 0.35 / thres_for_mean 0.48 / value 22.785

In [17]:
df_result_opt.head()

,rtn_3_fixed,rtn_4_fixed,rtn_5_fixed,rtn_3_float,rtn_4_float,rtn_5_float
date,,,,,,
2015-11-26,0.005756,0.030931,-0.015691,0.020728,0.025248,-0.003739
2015-12-23,0.009926,-0.001050,0.052341,-0.018486,-0.012281,0.009028
2016-01-22,-0.024512,0.012826,-0.073652,0.046095,0.004734,-0.092696
2016-02-23,0.084539,0.148932,0.159276,0.115318,0.126354,0.101664
2016-03-22,0.068666,0.063661,0.042678,0.081200,0.037448,0.067132


In [20]:
df_result_opt.to_pickle("df_oos_opt_result_applied.pkl")

In [28]:
(
    df_result_opt['rtn_5_fixed']
    + 1
).cumprod().plot(kind='scatter')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'variable=rtn_5_fixed<br>date=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'rtn_5_fixed',
              'marker': {'color': '#636efa', 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'rtn_5_fixed',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['2015-11-26', '2015-12-23', '2016-01-22', '2016-02-23', '2016-03-22',
                          '2016-04-19', '2016-05-18', '2016-06-15', '2016-07-12', '2016-08-08',
                          '2016-09-05', '2016-10-06', '2016-11-02', '2016-11-29', '2016-12-26',
                          '2017-01-23', '2017-02-21', '2017-03-21', '2017-04-17', '2017-05-18',
                          '2017-06-15', '2017-07-12', '2017-08-08', '2017-09-05', '2017-10-10',
                          '2017-11-06', '2017-12-01', '2018-01-02', '2018-01-29', '2018-02-27',
                          '2018-03-27', '2018-04-23', '2018-05-23', '2018-06-21', '2018-07-18',
                          '2018-08-14', '2018-09-11', '2018-10-15', '2018-11-09', '2018-12-06',
                          '2019-01-07', '2019-02-01', '2019-03-06', '2019-04-02', '2019-04-29',
                          '2019-05-28', '2019-06-25', '2019-07-22', '2019-08-19', '2019-09-17',
                          '2019-10-16', '2019-11-12', '2019-12-09', '2020-01-08', '2020-02-06',
                          '2020-03-04', '2020-03-31', '2020-04-28', '2020-05-28', '2020-06-24',
                          '2020-07-21', '2020-08-18', '2020-09-14', '2020-10-15', '2020-11-11',
                          '2020-12-08', '2021-01-07', '2021-02-03', '2021-03-05', '2021-04-01',
                          '2021-04-28', '2021-05-27', '2021-06-23', '2021-07-20', '2021-08-17',
                          '2021-09-13', '2021-10-15', '2021-11-11', '2021-12-08', '2022-01-05',
                          '2022-02-04', '2022-03-04', '2022-04-01', '2022-04-28', '2022-05-26',
                          '2022-06-24', '2022-07-21', '2022-08-18', '2022-09-16', '2022-10-17',
                          '2022-11-11', '2022-12-08', '2023-01-05', '2023-02-03', '2023-03-03',
                          '2023-03-30', '2023-04-26', '2023-05-25', '2023-06-23', '2023-07-20',
                          '2023-08-17', '2023-09-13', '2023-09-19'], dtype=object),
              'xaxis': 'x',
              'y': array([ 0.98430896,  1.03582888,  0.95953835,  1.11237007,  1.15984391,
                           1.25074617,  1.13864625,  1.26755176,  1.31720279,  1.36016892,
                           1.37332882,  1.45344931,  1.33399177,  1.41835029,  1.50780514,
                           1.54594496,  1.51923444,  1.58457723,  1.69964103,  1.59421842,
                           1.68362628,  1.73573572,  1.73022496,  1.7797188 ,  1.75994978,
                           1.89718766,  2.07832728,  2.04987348,  2.1440258 ,  2.09502832,
                           1.95720458,  2.25637144,  2.50211038,  2.42683454,  2.46790199,
                           2.58452939,  2.87268656,  2.46070216,  2.64100719,  3.03707024,
                           3.23856273,  3.52915304,  3.58014529,  3.65500218,  3.76678325,
                           3.75578104,  3.93406084,  3.82878679,  3.83181476,  4.30759448,
                           4.58536369,  4.96827682,  5.17870321,  5.39834544,  4.99000228,
                           4.64313266,  4.1469013 ,  4.59207355,  4.85577383,  4.7897342 ,
                           5.09266976,  5.57367325,  5.74990418,  5.43952025,  5.70622254,
                           6.07653422,  6.72621699,  6.90224984,  6.44904129,  7.3921436 ,
                           8.4029201 ,  8.97447483, 10.23006274, 10.66977047,  9.90485346,
                          10.66934151, 10.87266058, 10.22612837, 11.22216786, 11.40390769,
                          12.07158632, 12.58377678, 12.8570484 , 13.13889575, 13.02808487,
                          11.54352

In [34]:
# Check VaR
for rtn in ['rtn_3', 'rtn_4', 'rtn_5']:

    print(rtn)
    print(
        df_result_opt[f'{rtn}_fixed'].quantile([0, 0.05, 0.5, 0.95, 1])
    )

rtn_3
0.00   -0.141324
0.05   -0.091209
0.50    0.030010
0.95    0.130708
1.00    0.257797
Name: rtn_3_fixed, dtype: float64
rtn_4
0.00   -0.188093
0.05   -0.074883
0.50    0.034096
0.95    0.134820
1.00    0.234454
Name: rtn_4_fixed, dtype: float64
rtn_5
0.00   -0.143414
0.05   -0.081534
0.50    0.039171
0.95    0.139590
1.00    0.202111
Name: rtn_5_fixed, dtype: float64
